# Quick Runs: Running Modules

## NEQs = 6

In [2]:
# import numpy as np

# test = np.arange(-4.60971e-6,1e-6,0.5e-6)
# #print(type(test))
# print(test)
# print(len(test))

# current = [-4.60971e-6, 0, 4.7e-6, -6.7e-6, 6.7e-6, -4.7e-7, -4.7e-5]

# print(type(current))

# other = np.linspace(-4.60971e-6, 1e-6, num=20) 
# print(other)

# test = np.linspace(-4.60971e-6, 1e-6, num=19)  # 19 values
# test_with_zero = np.sort(np.append(test, 0.0))   # now 20 total
# print(test_with_zero)

In [5]:
# Core imports man
import sys
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Numerical libraries
from scipy.integrate import solve_ivp, odeint, cumulative_trapezoid
from scipy.interpolate import (
    UnivariateSpline, splrep, splev, CubicSpline, 
    interp1d, PchipInterpolator, InterpolatedUnivariateSpline
)

import numdifftools as nd

# Random number generator (GSL)
import pygsl.rng

# custom InflationModels code to path
sys.path.append(
    '/Users/epmeador/Desktop/research/rwarthur/inflation_gravitywaves/inflation_code/InflationModels'
)

# Enable spectra
SPECTRUM = True

# Local modules from InflationModels
from MacroDefinitions import *
from calcpath import *
from int_de import *
if SPECTRUM:
    from spectrum_old import *


# ========================
# GLOBAL SETTINGS
# ========================
NEQS = 6
SPECTRUM = True
SAVEPATHS = True

NMAX = 1.2
NMIN = 0.8

#lam3_set = [0.0, 4.95768e-6, -5e-6, 1e-6, -1e-6, 8e-6, 5e-7]
#lam3_set = [-4.60971e-6, 0, 4.7e-6, -6.7e-6, 6.7e-6, -4.7e-7, -4.7e-5] #what it used to be
#lam3_set = [np.arange(-4.60971e-6,1e-6,0.5e-6)] #what it is now
#lam3_set = np.linspace(-4.60971e-6, 1e-6, num=19) #19 models
#lam3_set = np.sort(np.append(lam3_set, 0.0))#now 20

lam3_set = np.linspace(-4.60971e-6, 1e-6, num=1) 


#This sets how many nontrivial models you are running (interesting models you want)
NUMPOINTS = 1        # number of nontrivial models per λ3

#This will give us the min and max number of e-folds we are looking for
NUMEFOLDSMAX = 65.0
NUMEFOLDSMIN = 57.0

# #Here we are writing out our output files we can name to analyze data

BASE_OUTDIR = "/Users/epmeador/Desktop/research/rwarthur/inflation_gravitywaves/inflation_code/Slow-Roll Parameters Tests/higgs_potential_tests/neqs6"

# RNG initialiation process
#This can be used to generate a random generator that works well with simulations

my_random = pygsl.rng.ranlxd2()
my_random.set(0)
np.random.seed(0)

# ========================
# Support Functions
# ========================

#Here we define a class for several variables, this will initialize several variables
#The size we are setting for Y and initY is the same as NEQs which may be number of equations and is set in flow.py
#We set a state, the initial state, an empty string in the class, number of points, and e-folds

class Calc:
    def __init__(self):
        self.Y = np.zeros(NEQS, dtype=float, order='C')
        self.initY = np.zeros(NEQS, dtype=float, order='C')
        self.ret = ""
        self.npoints = 0
        self.Nefolds = 0.0

#Below we are initializing our starting slow roll parameter values
#We should be able to choose what ell we are extending to.

def pick_init_vals(lam3):
#     init_vals = np.zeros(NEQS, dtype=float, order='C')
#     init_vals[0] = 6.0
#     init_vals[1] = 1.0
#     init_vals[2] = 0.000217261
#     init_vals[3] = -0.0345924
#     init_vals[4] = 0.000716771
#     init_vals[5] = lam3

    init_vals = np.zeros(NEQS, dtype=float, order='C')
    init_vals[0] = 5.5 #phi0
    init_vals[1] = 1.0 #H0
    init_vals[2] = 0.000209237 #epsilon0
    init_vals[3] = -0.0342419 #sigma0
    init_vals[4] = 0.000278972#2lam0
    init_vals[5] = lam3

    init_Nefolds = my_random.uniform() * (NUMEFOLDSMAX - NUMEFOLDSMIN) + NUMEFOLDSMIN
    return init_vals, init_Nefolds


#SAVING PATHS
#This next part of the code will print either asymptote or nontrivial 
#And decides when to save the code 
#This is based on what the model itself is doing

#This function below will calculation the spectrum computed from y
#As long as its in a particular range
#This is where I would limit my range of spectrum models of interest
#Otherwise it returns false

def we_should_calc_spec(y):
    return (specindex(y) > NMIN and specindex(y) < NMAX)

#Specifically, we will save a model with some interesting dynamics
#And this means either asymptote or nontrivial - for us nontrivial 
#Also only if the path has not been saved yet, and only for 
#Every nth successful model
#This governs path saving in the non-spectral case

def we_should_save_path(retval, save, pointcount, printevery):
    return (retval == "nontrivial") and (not save) and (pointcount % printevery == 0)

#Overall, this writes model trajectory to a file
#the SRP at each integration step, the number of N remaining, gives a reconstructed V, and e_H
    # Open output file

def save_path(y, N, kount, fname):
    with open(fname, "w") as outfile:
    # Output intermediate data from the integration
        for i in range(kount):
            for j in range(NEQS):
            #get y variable as a function of i in kount
            #should be like SRP as a function of time
            #j loops over NEQS which are used 
                outfile.write("%le " % y[j, i])
            outfile.write("%lf " % N[i])
            #Will also write out N at that time step
            V = 3 * y[1, i]**2 * (1. - y[2, i]/3.) #I think this is V #mpl^2=1
            outfile.write("%le %le\n" % (V, (V*y[2, i])/(3. - y[2, i]))) #I think this is KE

# ========================
# Main Loop
# ========================
def run_neqs6_models():
    summary_records = []
    

    for lam3 in lam3_set:
        print(f"\n=== Running λ3 = {lam3:.1e} ===")

        # Output directories
        OUTDIR = f"{BASE_OUTDIR}/lam3_{lam3:.1e}_original"
        os.makedirs(OUTDIR, exist_ok=True)
        OUTFILE1_NAME = f"{OUTDIR}/test_nr_neqs{NEQS}_original.dat"
        OUTFILE2_NAME = f"{OUTDIR}/test_esigma_neqs{NEQS}_original.dat"

        # Open output files for writing
        try:
            outfile1 = open(OUTFILE1_NAME, "w")
            outfile2 = open(OUTFILE2_NAME, "w")
        except IOError as e:
            print("Could not open output files: ", e)
            sys.exit()

            
        # Spectrum arrays
        if SPECTRUM:
            #Scalar mode function
            #u_s = np.empty((2, knos))
            u_s = np.empty((2, knos))
            #Tensor mode function
            #u_t = np.empty((2, knos))
            u_t = np.empty((2, knos))
            #This is an empty array of size NEQS + 1
            #This can be used to store slow roll variables plus some extra variable, maybe N 
            y_final = np.empty(NEQS + 1)
            #Below will track how many spectra have been saved
            spec_count = 0

            
        # Initialize counters and allocate buffers(?)
        #sets everthing to 0
        # iters = total number of iterations
        calc = Calc()
        iters = 0
        points = 0
        outcount = 0
        asymcount = 0
        nontrivcount = 0
        insuffcount = 0
        noconvcount = 0
        badncount = 0
        errcount = 0
        savedone = 0

        # Trial loop
        #Main part of MC loop: tries random models and decides whether to keep or discard them
        #This will keep looping as long as nontrivcount is less than NUMPOINTS
        #So it will find random inflation models until I have found enough nontrivial ones
        #nontrivial is an interesting model, if valid it will increment nontrivcount until you get NUMPOINTS
        #loop will stop once you get NUMPOINTS OR hit iters>200
        while nontrivcount < NUMPOINTS:
            iters += 1
            if iters > 200:  # failsafe
                break

         #this will print progress updates
         #every 100 iterations do something, every 1000 print status report
         #if multiple of 100 not 1000 print .
       
            if iters % 100 == 0:
                print(f"  Iter {iters}, nontriv={nontrivcount}")

            # pick initial conditions right now it is not random
            yinit, calc.Nefolds = pick_init_vals(lam3)
            #creates copy of random initial conditions created by pick_init_vals
            y = yinit.copy()

            # state arrays
            path = np.array([[]])
            N = np.array([])

            # integrate flow equations
            #this will call the calcpath function using random fluc copy, 
            #path from srp evolution, N, and calc 
            # calc stores state! So a string that categorizes this
            calc.ret = calcpath(calc.Nefolds, y, path, N, calc)
            print(f"  -> {calc.ret}")

            # handle outcome - i told it to get rid of asymptote
            #the original code kept them for late time attractors
            #if it is an asymptote we  proceed to find spectral indices and write them to a file
            #this is done after checking to see if specindex is between NMIN and NMAX
            #this is our nr file
            #this asymptote approached a late time attracror, could give valid observables, have to manually screen
            #mathematically consistent, exact solutions to flow,  usefulf or low-scale inflation models, can correspond to eternal inflation
            #they dont really allow for reheating which is why i dont like them
            #removed asymptote
            if calc.ret == "asymptote":
                asymcount += 1
                if asymcount > 100:
                    print("Too many asymptotes, stopping")
                    break
                continue


            #This block runs if nontrivial, which means it ended inflation after calc.Nefolds, not just asymptote
            #that block continues below
            if calc.ret == "nontrivial":
                # write observables
                r = tsratio(y)
                ns = specindex(y)
                alpha_s = dspecindex(y)
                outfile1.write(f"{r:.10f} {ns:.10f} {alpha_s:.10f}\n")
                outfile1.flush()

                # write final srp parameters and appends number
                for i in range(NEQS):
                    outfile2.write("%le " % y[i])
                outfile2.write("%f\n" % calc.Nefolds)
                outfile2.flush()

                points += 1 #total valid models saved so far
                savedone = 0 #marks that havent saved in teh trajectory files yet
                nontrivcount += 1 #counts how many nontrivial models we've accepted

                # calculate spectra if needed
                if SPECTRUM and we_should_calc_spec(y):
                    #this tells us what spectrum we are evaluating
                    print(f"  -> Evaluating spectrum {spec_count}")
                    #we will need to initalize ps evolution using slow roll values
                    #time step 3 so that its early enough in inflation so most modes are w/n horizon
                    y_final[:NEQS] = path[:NEQS, 3] #this grabs all slow roll parameters at time step 3
                    y_final[NEQS] = N[3] #this add the value of N at that time step
                    #dont forget : means slicing, path[:NEQS, 3] = path[0:NEQS,3]
                    #which is rows 0-NEQS-1, at column 3

                    #runs through spectrum module which uses that prepared final state
                    spectrum_status = spectrum(y_final, y, u_s, u_t, calc.Nefolds,
                                               derivs1, scalarsys, tensorsys)
                    if spectrum_status:
                        errcount += 1
                    #if something is wrong it will start counting errors

                    # save spectra
                    spec_s_name = f"{OUTDIR}/spec_s{spec_count:03d}_neqs{NEQS}_original.dat"
                    spec_t_name = f"{OUTDIR}/spec_t{spec_count:03d}_neqs{NEQS}_original.dat"
                    np.savetxt(spec_s_name, u_s[:, :knos].T)
                    np.savetxt(spec_t_name, u_t[:, :knos].T)
                    spec_count += 1

                    #Remember we are only computing power spectra for nontrivial models.
                    #For these models, inflation ends.
                    #To compute power spectrum you evolve mode k through horizon crossing k=aH
                    #And you evaluate perturbation amplitudes after horizon exit
                    #Attractor models never exit inflation

                #added a line in calcpath
                # normalize path if spectrum=True
#                 if SPECTRUM:
#                     for j in range(calc.npoints):
#                         #print("Final Hubble y[1] =", y[1])
#                         #this normalization shifts all phi vals
#                         #it subtracts the final phi from each one so last pt is 0
#                         #phi is recentered so end of inflation is at phi=0
#                         path[0, j] -= path[0, calc.npoints-1]
#                         #we then rescale the hubble values to get correct scale
#                         #if y[1] is less than zero this makes all H negative 
#                         #path[1, j] = path[1, j] * abs(y[1])
#                         path[1, j] *= abs(y[1])
                        
#                 #Added this        
#                 Hnorm = 0.00001 * 2 * np.pi * np.sqrt(y[2]) / y[1]
#                 path[1, :] *= Hnorm

                # save path
                path_name = f"{OUTDIR}/path_neqs{NEQS}_lam3{lam3:.1e}_{outcount:03d}_original.dat"
                print(f"  -> Saving path {path_name}")
                save_path(path, N, calc.npoints, path_name)
                outcount += 1

                # add summary record
                summary_records.append({
                    "lam3": lam3, "r": r, "n_s": ns,
                    "alpha_s": alpha_s, "Nefolds": calc.Nefolds
                })

            elif calc.ret == "insuff":
                insuffcount += 1
            elif calc.ret == "noconverge":
                noconvcount += 1
            else:
                errcount += 1

        # close output files
        outfile1.close()
        outfile2.close()

    # write summary CSV
    summary_df = pd.DataFrame(summary_records)
    summary_file = f"{BASE_OUTDIR}/neqs{NEQS}_summary_original.csv"
    summary_df.to_csv(summary_file, index=False)
    print(f"\nSummary written to {summary_file}")


if __name__ == "__main__":
    run_neqs6_models()

    
    


=== Running λ3 = -4.6e-06 ===
ε crosses 1 at step 24
  -> nontrivial
  -> Evaluating spectrum 0
Exiting spectrum evaluation normally
  -> Saving path /Users/epmeador/Desktop/research/rwarthur/inflation_gravitywaves/inflation_code/Slow-Roll Parameters Tests/higgs_potential_tests/neqs6/lam3_-4.6e-06_original/path_neqs6_lam3-4.6e-06_000_original.dat

Summary written to /Users/epmeador/Desktop/research/rwarthur/inflation_gravitywaves/inflation_code/Slow-Roll Parameters Tests/higgs_potential_tests/neqs6/neqs6_summary_original.csv


In [4]:
#This will run inside the notebook
#%prun runs profiler on the code
#-s time sorts results by total time spent in each function
#limits output to top 20 function
#call the function you wanna use which for me is spectrum and requires the arg

%prun -s cumtime run_neqs6_models()


=== Running λ3 = -4.6e-06 ===
ε crosses 1 at step 24
  -> nontrivial
  -> Evaluating spectrum 0
Exiting spectrum evaluation normally
  -> Saving path /Users/epmeador/Desktop/research/rwarthur/inflation_gravitywaves/inflation_code/Slow-Roll Parameters Tests/higgs_potential_tests/neqs6/lam3_-4.6e-06_current/path_neqs6_lam3-4.6e-06_000_current.dat

Summary written to /Users/epmeador/Desktop/research/rwarthur/inflation_gravitywaves/inflation_code/Slow-Roll Parameters Tests/higgs_potential_tests/neqs6/neqs6_summary_current.csv
 